<a href="https://colab.research.google.com/github/AlibekAdilzhan/ml_models/blob/main/my_adaboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('sample_data/california_housing_train.csv')
df = df.dropna()
df.describe()
df_test = df[15000 : 17000]
df = df[ : 15000]
# np.where(np.isnan(df))
df.describe()

In [ ]:
X_train = df.drop(["median_house_value"], axis = 1)
Y_train = df["median_house_value"]
# type(X_train["Age"])
X_train

In [ ]:
Y_train = Y_train.apply(lambda x: 1 if x > 350000 else -1)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from copy import deepcopy

In [ ]:
def my_adaboost(X_train, Y_train, model_number):
    models_list = []
    alphas_list = []
    # model_number = 30
    X_train['W'] = [1 / len(X_train)] * len(X_train)
    X = X_train.copy()
    Y = Y_train.copy()

    for i in range(model_number):
        tree_model = DecisionTreeClassifier(max_depth = 3)
        tree_model.fit(X.drop(['W'], axis = 1), Y, sample_weight = X['W'])
        models_list.append(deepcopy(tree_model))

        Y_predict = tree_model.predict(X.drop(['W'], axis = 1))
        # print(Y_predict)
        right_indices = (Y_predict != Y).astype(int)
        weights = X['W'] * right_indices
        epsilon = np.sum(weights)

        alpha = 1 / 2 * np.log((1 - epsilon) / epsilon)
        alphas_list.append(alpha)

        for i in range(len(X['W'])):
            if right_indices[i] == 1:
                X['W'][i] *= np.exp(-Y[i] * alpha * Y_predict[i])

        X['W'] /= np.sum(np.array(X['W']))

    return models_list, alphas_list
    
    

In [ ]:
models_list, aphas_list = my_adaboost(X_train, Y_train, 30)

In [ ]:
# df_test = pd.read_csv('sample_data/california_housing_test.csv')
X_test = df_test.drop('median_house_value', axis = 1)
Y_test = df_test['median_house_value']
Y_test = Y_test.apply(lambda x: 1 if x > 350000 else -1)
Y_test

In [ ]:
prediction = np.zeros(Y_test.shape).astype(float)

print(prediction)
for i in range(len(models_list)):
    model = models_list[i]
    alpha = aphas_list[i]

    prediction += alpha * model.predict(X_test).T

# prediction = (prediction > 0).astype(int)
prediction

In [ ]:
prediction_category = np.zeros(prediction.shape)

for i in range(len(Y_test)):
    if prediction[i] >= 0:
        prediction_category[i] = 1
    else:
        prediction_category[i] = -1

In [ ]:
results = (prediction_category == Y_test).astype(int)
print(np.sum(results))

In [ ]:
print(len(Y_test))
Y_test_array = np.array(Y_test.copy())

In [ ]:
print(np.sum(results) / len(Y_test))


In [ ]:
print(results)

In [ ]:
print(Y_test)

In [ ]:
print(prediction_category)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
aBoost = AdaBoostClassifier(n_estimators = 30, learning_rate = 0.5)
aBoost.fit(X_train.drop(['W'], axis = 1), Y_train)

In [ ]:
prediction_boost = aBoost.predict(X_test)
prediction_boost

In [ ]:
results_boost = (prediction_boost == Y_test).astype(int)

print(np.sum(results_boost) / len(Y_test))
